<a href="https://colab.research.google.com/github/YKulki/DE_HUL/blob/main/DE_ACGIH_Components.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving DE Calculations - BC (1).csv to DE Calculations - BC (1).csv


In [ ]:
import io
import pandas as pd

df = pd.read_csv(io.BytesIO(uploaded['DE Calculations - BC (1).csv']))
df['Tag No.'] = df['Tag No.'].astype('string')
df['Powder Type'] = df['Powder Type'].astype('string')
df['Belt Drop Heights Inlet(mm)'] = df['Belt Drop Heights Inlet(mm)'].astype('string')
df['Outlet Areas(m2)'] = df['Outlet Areas(m2)'].astype('string')
df['Opening Area(m2)'] = df['Opening Area(m2)'].astype('string')

In [ ]:
def belt_conveyor(length,width,speed,inlets,outlets,drop_heights,areas,powder_flow_rate,dust_gen_factor):

  inlet_q=[]
  length_q=[]
  outlet_q=[]


  #emp_factor=float(input("Material dust correction factor"))

  width=0.00328084*width #in mm  
  speed=196.85*speed  #in m/s


  length=3.28084*length #in m
  heights=list(map(float, drop_heights.split(',')))


  for x in range(inlets):
    drop_height=heights[x]
    if drop_height<0:
      inlet_q.append(0)
    else:
      if drop_height<0.9144:
        if speed <200:
          inlet_q.append(594.6536*width)
        else:
          inlet_q.append(849.5051*width)
      else:
        if speed<200:
          inlet_q.append(594.6536*width)
        else:
          inlet_q.append(849.5051*width)
        if width<3:
          inlet_q.append(1189.3071)
        else:
          inlet_q.append(1699.0102)


  n=int(length/30)
  for x in range(n):
    length_q.append(594.6536*width)

  outlet_areas=list(map(float, areas.split(',')))

  for x in range(outlets):
    outlet_area=10.7639*outlet_areas[x]   #in m2
    outlet_q.append(339.802*outlet_area)

  
  q_powder_flow=dust_gen_factor*powder_flow_rate

  if sum(inlet_q)+sum(length_q)+sum(outlet_q)<q_powder_flow:
    print("Check with emperical formula")
  else:  
    Inlet_Q = ",".join(map(str, inlet_q))
    Outlet_Q =",".join(map(str, outlet_q))
    Length_Q =",".join(map(str, length_q))



    return [Inlet_Q,Outlet_Q,Length_Q]

In [ ]:
def vessel(openings,volume,hopper_type,empty_time,cycles,opening_area,powder_flow_rate,dust_gen_factor):

  q_calc=[]

  opening_area=list(map(float, opening_area.split(',')))

  if empty_time>0 and cycles>0 and volume>0:
    if hopper_type=='B':
      q_airchange=volume*60*cycles/empty_time #cycles per min,empty time in min
    elif hopper_type=='D':
      #8-15 air changes recommended
      q_airchange=volume*60*cycles/empty_time #cycles per min,empty time in min
    else:
      print("Incorrect input")
  else: 
    q_airchange=0

  for x in range(openings):
    if opening_area[x]>0:
      area=10.7639*opening_area[x] #in m^2
      q_calc.append(339.8*area)
    else:
      q_calc.append(0)

  q_powder_flow=float(dust_gen_factor*powder_flow_rate)

  return max(sum(q_calc),q_airchange,q_powder_flow)




In [ ]:
 def Sieve(sieve_area):

  sieve_area=10.7639*sieve_area

  q=1.699*50*sieve_area

  q_powder_flow=float(dust_gen_factor*powder_flow_rate)

  return max(q,q_powder_flow)

In [ ]:
print(df.dtypes)

Tag No.                         string
Powder Type                     string
Dust_gen_factor                  int64
Powder Flow Rate(m3/hr)          int64
Inlets                         float64
Outlets                        float64
Length(m)                      float64
Belt Width(mm)                 float64
Belt Speed (m/s)               float64
Belt Drop Heights Inlet(mm)     string
Outlet Areas(m2)                string
Hopper Type                     object
Openings                       float64
Opening Area(m2)                string
Volume(m3)                     float64
Empty Time(min)                float64
Cycles (per min)               float64
Sieve_Area(m2)                 float64
Inlet_Q                        float64
Outlet_Q                       float64
Length_Q                       float64
Vessel_Q                       float64
Sieve_Q                        float64
dtype: object


In [ ]:
q=[]

for ind in df.index:
  if not pd.isnull(df.loc[ind, 'Tag No.']): 
    if df['Tag No.'][ind][:2]== "BC":
      length=df['Length(m)'][ind]
      width=df['Belt Width(mm)'][ind]
      speed=float(df['Belt Speed (m/s)'][ind])
      inlets=int(df['Inlets'][ind])
      outlets=int(df['Outlets'][ind])
      drop_heights=df['Belt Drop Heights Inlet(mm)'][ind]
      areas=df['Outlet Areas(m2)'][ind]
      powder_flow_rate=df['Powder Flow Rate(m3/hr)'][ind]
      dust_gen_factor=df['Dust_gen_factor'][ind]
      q=(belt_conveyor(length,width,speed,inlets,outlets,drop_heights,areas,powder_flow_rate,dust_gen_factor))
    elif df['Tag No.'][ind][:2]== "WH"or df['Tag No.'][ind][:2]== "DH" or df['Tag No.'][ind][:2]== "VS":
      openings=int(df['Openings'][ind])
      opening_area=df['Opening Area(m2)'][ind] 
      volume=df['Volume(m3)'][ind]
      hopper_type=df['Hopper Type'][ind]
      empty_time=df['Empty Time(min)'][ind]
      cycles=df['Cycles (per min)'][ind]
      powder_flow_rate=df['Powder Flow Rate(m3/hr)'][ind]
      dust_gen_factor=df['Dust_gen_factor'][ind]
      q=vessel(openings,volume,hopper_type,empty_time,cycles,opening_area,powder_flow_rate,dust_gen_factor)
    elif df['Tag No.'][ind][:1]=="S":
      sieve_area=float(df['Sieve_Area(m2)'][ind])
      q=Sieve(sieve_area)
    else:
      print("Soz")
  if df['Tag No.'][ind][:2]== "BC":
    df['Inlet_Q'][ind]=q[0]
    df['Outlet_Q'][ind]=q[1]
    df['Length_Q'][ind]=q[2]
  elif df['Tag No.'][ind][:2]== "WH"or df['Tag No.'][ind][:2]== "DH" or df['Tag No.'][ind][:2]== "VS":
    df['Vessel_Q'][ind]=q
  elif df['Tag No.'][ind][:1]=="S":
    df['Sieve_Q'][ind]=q
  else:
    print("soz")  
      




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher

In [ ]:
df


,Tag No.,Powder Type,Dust_gen_factor,Powder Flow Rate(m3/hr),Inlets,Outlets,Length(m),Belt Width(mm),Belt Speed (m/s),Belt Drop Heights Inlet(mm),...,Opening Area(m2),Volume(m3),Empty Time(min),Cycles (per min),Sieve_Area(m2),Inlet_Q,Outlet_Q,Length_Q,Vessel_Q,Sieve_Q
0,BC-5315,bp,1,107,3.0,1.0,19.230,1200.0,0.450000,"10,0.332,2.765",...,<NA>,NaN,NaN,NaN,NaN,"2341.1559804288,1699.0102,2341.1559804288,2341...",914.398687,"2341.1559804288,2341.1559804288",NaN,NaN
1,BC-5625,bp,1,112,2.0,1.0,9.625,1200.0,0.450000,"2.951,2.412",...,<NA>,NaN,NaN,NaN,NaN,"2341.1559804288,1699.0102,2341.1559804288,1699...",914.398687,2341.1559804288,NaN,NaN
2,BC-5680,bp,1,112,1.0,2.0,9.717,1200.0,0.450000,0.988,...,<NA>,NaN,NaN,NaN,NaN,"2341.1559804288,1699.0102","914.3986869500001,914.3986869500001",2341.1559804288,NaN,NaN
3,BC-5735,bp,1,65,1.0,1.0,5.840,1200.0,0.333333,-1,...,<NA>,NaN,NaN,NaN,NaN,0,914.3986869500001,,NaN,NaN
4,BC-5745,bp,1,65,1.0,1.0,5.840,1200.0,0.333333,-1,...,<NA>,NaN,NaN,NaN,NaN,0,914.3986869500001,,NaN,NaN
5,BC-5825,bp,1,132,1.0,1.0,32.760,1200.0,0.600000,-1,...,<NA>,NaN,NaN,NaN,NaN,0,1463.03789912,"2341.1559804288,2341.1559804288,2341.1559804288",NaN,NaN
6,BC-5870,bp,1,132,1.0,2.0,4.977,1200.0,0.600000,1.042,...,<NA>,NaN,NaN,NaN,NaN,"2341.1559804288,1699.0102","585.215159648,585.215159648",,NaN,NaN
7,BC-5880,bp,1,132,1.0,2.0,4.977,1200.0,0.600000,1.042,...,<NA>,NaN,NaN,NaN,NaN,"2341.1559804288,1699.0102","585.215159648,585.215159648",,NaN,NaN
8,BC-7585,pd,1,100,11.0,1.0,41.140,1200.0,0.600000,"2.45,2.45,2.45,2.45,2.45,2.45,2.45,2.45,2.45,2...",...,<NA>,NaN,NaN,NaN,NaN,"2341.1559804288,1699.0102,2341.1559804288,1699...",1463.03789912,"2341.1559804288,2341.1559804288,2341.155980428...",NaN,NaN
9,BC-7880,pd,1,100,1.0,2.0,4.097,1200.0,0.600000,2.57,...,<NA>,NaN,NaN,NaN,NaN,"2341.1559804288,1699.0102","1645.91763651,1645.91763651",,NaN,NaN


In [ ]:
df.to_csv('df.csv')
files.download('df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

0.06683517070054885
